In [2]:
import findspark
findspark.init('/usr/local/spark')
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('lrex').getOrCreate()

In [3]:
from pyspark.ml.regression import LinearRegression

In [5]:
training = spark.read.format('libsvm').load('Data/Spark_for_Machine_Learning/Linear_Regression/sample_linear_regression_data.txt')

In [6]:
training.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -9.490009878824548|(10,[0,1,2,3,4,5,...|
| 0.2577820163584905|(10,[0,1,2,3,4,5,...|
| -4.438869807456516|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -7.966593841555266|(10,[0,1,2,3,4,5,...|
| -7.896274316726144|(10,[0,1,2,3,4,5,...|
| -8.464803554195287|(10,[0,1,2,3,4,5,...|
| 2.1214592666251364|(10,[0,1,2,3,4,5,...|
| 1.0720117616524107|(10,[0,1,2,3,4,5,...|
|-13.772441561702871|(10,[0,1,2,3,4,5,...|
| -5.082010756207233|(10,[0,1,2,3,4,5,...|
|  7.887786536531237|(10,[0,1,2,3,4,5,...|
| 14.323146365332388|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-0.8995693247765151|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|  5.601801561245534|(10,[0,1,2,3,4,5,...|
|-3.2256352187273354|(10,[0,1,2,3,4,5,...|
| 1.5299675726687754|(10,[0,1,2,3,4,5,...|
| -0.250102447941961|(10,[0,1,2,3,4,5,...|
+----------

In [7]:
lr = LinearRegression(featuresCol='features', labelCol='label', predictionCol='prediction')

In [8]:
lrModel = lr.fit(training)

In [10]:
lrModel.coefficients

DenseVector([0.0073, 0.8314, -0.8095, 2.4412, 0.5192, 1.1535, -0.2989, -0.5129, -0.6197, 0.6956])

In [11]:
lrModel.intercept

0.14228558260358093

In [12]:
training_summary = lrModel.summary

In [13]:
training_summary.r2

0.027839179518600154

In [14]:
training_summary.rootMeanSquaredError

10.16309157133015

In [15]:
all_data = spark.read.format('libsvm').load('Data/Spark_for_Machine_Learning/Linear_Regression/sample_linear_regression_data.txt')

In [18]:
train_data, test_data = all_data.randomSplit([0.7, 0.3])

In [19]:
train_data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                369|
|   mean|-0.3943215701231476|
| stddev| 10.428736132111354|
|    min|-28.571478869743427|
|    max| 27.111027963108548|
+-------+-------------------+



In [20]:
correct_model = lr.fit(train_data)

In [21]:
test_results = correct_model.evaluate(test_data)

In [23]:
test_results.predictions.show()

+-------------------+--------------------+--------------------+
|              label|            features|          prediction|
+-------------------+--------------------+--------------------+
|-23.487440120936512|(10,[0,1,2,3,4,5,...| -2.5139342560326234|
|-19.872991038068406|(10,[0,1,2,3,4,5,...|  -0.499731069659496|
|-17.494200356883344|(10,[0,1,2,3,4,5,...| -1.7991758305078451|
| -16.26143027545273|(10,[0,1,2,3,4,5,...|   4.017698550855307|
| -15.72351561304857|(10,[0,1,2,3,4,5,...|  1.9132347965366976|
|-15.437384793431217|(10,[0,1,2,3,4,5,...| -2.9810758421009127|
|-15.334767479922341|(10,[0,1,2,3,4,5,...|  2.0023981199644547|
| -13.15333560636553|(10,[0,1,2,3,4,5,...| -2.9849965759611186|
|-12.977848725392104|(10,[0,1,2,3,4,5,...|-0.15106236266020256|
|-12.500773785355054|(10,[0,1,2,3,4,5,...| 0.17586323810541327|
|-12.491442077546413|(10,[0,1,2,3,4,5,...|  0.7576759178947032|
|-11.827072996392571|(10,[0,1,2,3,4,5,...|  2.6163223049541338|
|-11.615775265015627|(10,[0,1,2,3,4,5,..

# Code Along

In [26]:
spark = SparkSession.builder.appName('lr_example').getOrCreate()

In [30]:
data = spark.read.csv('Data/Spark_for_Machine_Learning/Linear_Regression/Ecommerce_Customers.csv', inferSchema=True,
                      header=True)

In [31]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [32]:
data.show()

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512|37.110597442120856|   4.104543202376424| 487.54750486747207|
|riverarebecca@gma...|1414 David Throug...|   

In [33]:
for item in data.head(1)[0]:
    print(item)

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.49726772511229
12.65565114916675
39.57766801952616
4.0826206329529615
587.9510539684005


In [34]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [37]:
input_cols = ['Avg Session Length', 'Time on App', 'Time on Website', 'Length of Membership', 'Yearly Amount Spent']
assembler = VectorAssembler(inputCols=input_cols, outputCol='features')

In [38]:
output = assembler.transform(data)

In [42]:
# not necessary but cleaner
final_data = output.select('features', 'Yearly Amount Spent')

In [43]:
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [44]:
train_data, test_data = final_data.randomSplit([0.7, 0.3])

In [45]:
lr = LinearRegression(labelCol='Yearly Amount Spent')

In [46]:
lr_model = lr.fit(train_data)

In [47]:
test_result = lr_model.evaluate(test_data)

In [50]:
test_result.residuals.show()

+--------------------+
|           residuals|
+--------------------+
|1.875832822406664...|
|-2.10320649784989...|
|-9.66338120633736...|
|1.136868377216160...|
|-7.38964445190504...|
|-6.25277607468888...|
|1.193711796076968...|
|7.958078640513122...|
|-1.02318153949454...|
|1.648459146963432...|
|3.296918293926865...|
|-9.09494701772928...|
|-8.52651282912120...|
|-9.09494701772928...|
|5.684341886080801...|
|3.069544618483633...|
|-1.70530256582424...|
|-1.13686837721616...|
|-3.12638803734444...|
|1.421085471520200...|
+--------------------+
only showing top 20 rows



In [51]:
test_results.r2

-0.17489218084653935

In [52]:
test_results.rootMeanSquaredError

10.59519945635557

In [54]:
final_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                500|
|   mean|  499.3140382585909|
| stddev|   79.3147815497068|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [55]:
unlabeled_data = test_data.select('features')

In [56]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|[29.5324289670579...|
|[30.3931845423455...|
|[30.4925366965402...|
|[30.5743636841713...|
|[30.8364326747734...|
|[30.8794843441274...|
|[30.9716756438877...|
|[31.0472221394875...|
|[31.0613251567161...|
|[31.1695067987115...|
|[31.3123495994443...|
|[31.5147378578019...|
|[31.5171218025062...|
|[31.5257524169682...|
|[31.5316044825729...|
|[31.6005122003032...|
|[31.6610498227460...|
|[31.7207699002873...|
|[31.8164283341993...|
|[31.8512531286083...|
+--------------------+
only showing top 20 rows



In [57]:
predictions = lr_model.transform(unlabeled_data)

In [58]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[29.5324289670579...| 408.6403510726256|
|[30.3931845423455...| 319.9288698031957|
|[30.4925366965402...| 282.4712457199155|
|[30.5743636841713...|442.06441375806554|
|[30.8364326747734...|467.50190042699035|
|[30.8794843441274...| 490.2065999848553|
|[30.9716756438877...| 494.6386097568915|
|[31.0472221394875...| 392.4973991890206|
|[31.0613251567161...| 487.5554580579026|
|[31.1695067987115...|427.35653080229116|
|[31.3123495994443...| 463.5914180279373|
|[31.5147378578019...| 489.8124879964623|
|[31.5171218025062...|275.91842065038657|
|[31.5257524169682...| 443.9656268098828|
|[31.5316044825729...|  436.515605729362|
|[31.6005122003032...|479.17285149109387|
|[31.6610498227460...|  416.358353579901|
|[31.7207699002873...| 538.7749334780241|
|[31.8164283341993...| 501.1224915036595|
|[31.8512531286083...|  472.992246666797|
+--------------------+------------

In [6]:
spark = SparkSession.builder.appName('lr_example').getOrCreate()

In [8]:
data = spark.read.csv('Data/Spark_for_Machine_Learning/Linear_Regression/Ecommerce_Customers.csv',
                      inferSchema=True, header=True)

In [ ]:
data.